In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import tensorflow as tf
import sys
sys.path.append('../bmpc_shared_scripts/refinement_transfer_learning')
sys.path.append('../bmpc_shared_scripts/oktoberfest_interface')
from oktoberfest_interface import load_keras_model, process_dataset
from automatic_rl_tl import AutomaticRlTlTraining, AutomaticRlTlTrainingConfig

2024-07-24 15:56:17.022077: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-24 15:56:17.022120: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-24 15:56:17.023306: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-24 15:56:17.030080: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-07-24 15:56:18.924557: W tensorflow/compiler/tf2


Avaliable feature extractors are (use the key of the following dict and pass it to features_to_extract in the Dataset Class):
{
   "atom_count": "Atom count of PTM.",
   "delta_mass": "Delta mass of PTM.",
   "mod_gain": "Gain of atoms due to PTM.",
   "mod_loss": "Loss of atoms due to PTM.",
   "red_smiles": "Reduced SMILES representation of PTM."
}.
When writing your own feature extractor, you can either
    (1) use the FeatureExtractor class or
    (2) write a function that can be mapped to the Hugging Face dataset.
In both cases, you can access the parsed sequence information from the dataset using the following keys, which all provide python lists:
    - _parsed_sequence: parsed sequence
    - _n_term_mods: N-terminal modifications
    - _c_term_mods: C-terminal modifications



In [3]:
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
os.environ['HF_HOME'] = '/cmnfs/proj/bmpc_dlomix/datasets'
os.environ['HF_DATASETS_CACHE'] = '/cmnfs/proj/bmpc_dlomix/datasets/hf_cache'

num_proc = 16
os.environ["OMP_NUM_THREADS"] = f"{num_proc}"
os.environ["TF_NUM_INTRAOP_THREADS"] = f"{num_proc}"
os.environ["TF_NUM_INTEROP_THREADS"] = f"{num_proc}"

tf.config.threading.set_inter_op_parallelism_threads(num_proc)
tf.config.threading.set_intra_op_parallelism_threads(num_proc)

In [11]:
# small dataset
# parquet_path = '/cmnfs/data/proteomics/Prosit_PTMs/21PTMs/Kmod_Formyl.parquet'
# modifications = ['K[UNIMOD:122]']
# large single ptm
parquet_path= '/cmnfs/data/proteomics/Prosit_PTMs/TUM_mod_monomethyl.parquet'
modifications = ['K[UNIMOD:34]', 'R[UNIMOD:34]']
ion_types = ['y', 'b']

In [5]:
# load baseline model
model = load_keras_model('baseline')
model.summary()

Using cached model: /cmnfs/home/f.kapitza/.dlomix/models/prosit_baseline_model.keras


2024-07-24 15:56:24.767294: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 7505 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:82:00.0, compute capability: 6.1


Model: "prosit_intensity_predictor"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       multiple                  464       
                                                                 
 sequential (Sequential)     (None, 30, 512)           1996800   
                                                                 
 sequential_1 (Sequential)   multiple                  4608      
                                                                 
 sequential_2 (Sequential)   (None, 29, 512)           1576806   
                                                                 
 encoder_att (AttentionLaye  multiple                  542       
 r)                                                              
                                                                 
 sequential_3 (Sequential)   multiple                  0         
                                        

In [12]:
dataset = process_dataset(
    parquet_file_path=parquet_path,
    model=model,
    modifications=modifications,
    ion_types=ion_types,
    label_column='intensities_raw',
    val_ratio=0.2
)

/cmnfs/home/f.kapitza/dlomix/finn_notebooks/../bmpc_shared_scripts/oktoberfest_interface/oktoberfest_interface.py:134: UserWarning: 
            There are new tokens in the dataset, which are not supported by the loaded model.
            Either load a different model or transfer learning needs to be done.
            
  warnings.warn(


Start processing the dataset...


Generating train split: 335479 examples [00:13, 24372.80 examples/s]
Casting the dataset: 100%|██████████| 67096/67096 [00:12<00:00, 5330.20 examples/s]


The available data splits are: train, val


In [13]:
config = AutomaticRlTlTrainingConfig(
    dataset=dataset,
    baseline_model=model,
    use_wandb=True
)

In [14]:
trainer = AutomaticRlTlTraining(config)

[embedding layer]  model and dataset modifications do not match
[embedding layer]  can reuse old embedding weights
[regressor layer]  matching ion types


In [15]:
new_model = trainer.train()

1000/1000 [==============================] - 15s 15ms/step - loss: 0.7488 - masked_pearson_correlation_distance: 0.6686
validation loss: 0.7488385438919067, pearson distance: 0.668613612651825
Epoch 1/10015
4194/4194 [==============================] - 78s 18ms/step - loss: 0.7640 - masked_pearson_correlation_distance: 0.6984 - val_loss: 0.7069 - val_masked_pearson_correlation_distance: 0.6018
Epoch 2/10015
4194/4194 [==============================] - 69s 17ms/step - loss: 0.7633 - masked_pearson_correlation_distance: 0.6977 - val_loss: 0.7048 - val_masked_pearson_correlation_distance: 0.5994
Epoch 3/10015
4194/4194 [==============================] - 71s 17ms/step - loss: 0.7608 - masked_pearson_correlation_distance: 0.6949 - val_loss: 0.7005 - val_masked_pearson_correlation_distance: 0.5942
Epoch 4/10015
4194/4194 [==============================] - 72s 17ms/step - loss: 0.7559 - masked_pearson_correlation_distance: 0.6897 - val_loss: 0.6914 - val_masked_pearson_correlation_distance: 0.

epoch,▁▁▂▂▃▃▄▄▅▅▆▇▇██▁▁▂▂▃▄▄▅▅▆▆▇▇██▁▁▂▂▃▃▄▄▅▆
epoch_total,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
es_avg_change,████▇▇▆▅▃▂▂▁█████▇▇▆▅▃▁▁████████████████
es_curr_change,████▇▆▅▅▄▄▄▅█████▇▇▅▄▂▁▆████████████████
freeze_embedding_layer,▁▁▁
freeze_inner_layers,█▁▁
freeze_old_embedding_weights,█▁▁
freeze_old_regressor_weights,▁▁▁
freeze_regressor_layer,█▁▁
learning_rate,▁▁▁▁▁▁▁▁▁▁▁▂▃▇█▁▁▁▁▁▁▁▁▁▁▂▃▄███▂▂▂▂▁▁▁▁▁
loss,██████▇▇▇▆▆▆▆▆▆▆▆▆▆▆▆▆▅▅▄▄▃▂▂▂▂▁▁▁▁▁▁▁▁▁


In [10]:
import wandb
wandb.finish()

epoch,▁▅█
epoch_total,▁▃▆█
es_avg_change,▁▅▆▇▇███████████████████████████████████
es_curr_change,▁▅▆▇▇███████████████████████████████████
freeze_embedding_layer,▁
freeze_inner_layers,▁
freeze_old_embedding_weights,▁
freeze_old_regressor_weights,▁
freeze_regressor_layer,▁
learning_rate,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,▄▆▇▇▇▇█▇▇▇▇▆▇▇▇███▇▇▇▄▆▇████▇▇▆▁▅▆▇▇▇█▇▇
